## Import necessary libraries

In [27]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from costs import *
from models import *
from helpers import * 
from evaluation import *
from split_data import *

## Preprocessing
** Load the training data into feature matrix, class labels, and record ids**

We write our own `load_csv_data` function to import csv data, which gives us prediction column, feature matrix and each record ID.

In [54]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tx, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=False)

** Split into 6 distinct datasets**
According to our exploration, we can distinct 3 different dataset based on number of jets each experiments contains. Then each of them can be split again into 2 different datasets based on whether they have a measurable mass or not.

In [55]:
jet0, jet1, jet23, y0, y1, y23 = split_on_jets(y, tx)
jet0_nomass, jet0, y0_nomass, y0 = split_on_mass(y0, jet0)
jet1_nomass, jet1, y1_nomass, y1 = split_on_mass(y1, jet1)
jet23_nomass, jet23, y23_nomass, y23 = split_on_mass(y23, jet23)

** Standardization of values**
We use [feature scaling](https://en.wikipedia.org/wiki/Feature_scaling) method to standardize our feature matrix, i.e. to rescale tx down to [0, 1], so as to avoid complicated computation caused by large numbers.

In [56]:
tx = standardize(tx)
jet0_nomass = standardize(jet0_nomass)
jet0 = standardize(jet0)
jet1_nomass = standardize(jet1_nomass)
jet1 = standardize(jet1)
jet23_nomass = standardize(jet23_nomass)
jet23 = standardize(jet23)

## Model Selection

Let's begin with a simple linear regression with least_square using **normal equations**. Here we don't consider using least squares with gradient descent or stochastic gradient descent for the fact that **optimal w could be derived thoeritically**. We therefore don't bother to estimate the w.

We run cross validation 4 times on our train_data to see LS performance

### Logistic Regression

Choose intial parameters

In [57]:
n_iters = 2000
gamma = 0.000003

Train with logistic regression

In [60]:
print("w0_nomass")
loss0_nomass, w0_nomass = logistic_regression(y0_nomass, jet0_nomass, gamma, n_iters)
print("w0")
loss0, w0 = logistic_regression(y0, jet0, gamma, n_iters)

print("w1_nomass")
loss1_nomass, w1_nomass = logistic_regression(y1_nomass, jet1_nomass, gamma, n_iters)
print("w1")
loss1_nomass, w1 = logistic_regression(y1, jet1, gamma, n_iters)

print("w23_nomass")
loss23_nomass, w23_nomass = logistic_regression(y23_nomass, jet23_nomass, gamma, n_iters)
print("w23")
loss23, w23 = logistic_regression(y23, jet23, gamma, n_iters)


w0_nomass
Current iteration=0, the loss=18107.08379776745, gradient=0.24041936232171168
Current iteration=100, the loss=18110.495402926295, gradient=0.2392022137219361
Current iteration=200, the loss=18113.890301224466, gradient=0.23799122887558502
Current iteration=300, the loss=18117.268571521756, gradient=0.23678637657804344
Current iteration=400, the loss=18120.63029233654, gradient=0.23558762578271833
Current iteration=500, the loss=18123.97554184695, gradient=0.23439494560024096
Current iteration=600, the loss=18127.30439789197, gradient=0.2332083052976695
Current iteration=700, the loss=18130.616937972623, gradient=0.23202767429769758
Current iteration=800, the loss=18133.913239253096, gradient=0.23085302217786627
Current iteration=900, the loss=18137.1933785619, gradient=0.2296843186697798
Current iteration=1000, the loss=18140.457432393, gradient=0.22852153365832648
Current iteration=1100, the loss=18143.705476906947, gradient=0.22736463718090183
Current iteration=1200, the lo

In [ ]:
cross_validation(y, tx, 4, 0, logistic_regression, 0.000003)

Predict with our trained model

In [61]:
test_x = np.genfromtxt('data/test.csv', delimiter=',', skip_header=1)

In [62]:
ids = test_x[:, 0]
testset = standardize(test_x[:, 2:])  # remove id and prediction columns

In [65]:
prediction = apply_right_model(testset, ids, w0_nomass, w0, w1_nomass, w1, w23_nomass, w23)
prediction

array([[  3.50000000e+05,   1.00000000e+00],
       [  3.50001000e+05,   1.00000000e+00],
       [  3.50002000e+05,   1.00000000e+00],
       ..., 
       [  9.18235000e+05,   1.00000000e+00],
       [  9.18236000e+05,   1.00000000e+00],
       [  9.18237000e+05,   1.00000000e+00]])

In [66]:
print("higgs: ", np.count_nonzero(prediction == 1))
print("non-higgs: ", np.count_nonzero(prediction == 0))

higgs:  568238
non-higgs:  0


In [ ]:
test_x = np.genfromtxt('data/test.csv', delimiter=',', skip_header=1)
test_x = standardize(test_x[:, 2:])  # remove id and prediction columns
# could've used load_csv_data
create_csv_submission([i for i in range(350000,918238)], log_reg_predict(test_x, w), 'res.csv')

## Feature Engineering
TODO

## Prediction

**Generate predictions and save ouput in csv format for submission**